In [1]:
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.model_selection import train_test_split
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import os
import numpy as np
import mediapipe as mp
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
import cv2


In [2]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode = True, min_detection_confidence = 0.3)

In [3]:
def load_data(path):
    images = []
    labels = []
    labels_idx = []
    landmarks = []
    unique_labels=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34'] 
    for idx, label in tqdm(enumerate(unique_labels), total=len(unique_labels)):
        for filename in os.listdir(path + "/" + label):
            image_path = os.path.join(path, label, filename)

            # Get hand landmarks
            landmark_coords = []
            
            im = cv2.imread(image_path)
            img_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            
            results = hands.process(img_rgb)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    for i in range(len(hand_landmarks.landmark)):
                        
                        landmark = hand_landmarks.landmark[i]
                        

                        # Get coordinates
                        landmark_coords.append(landmark.x)
                        landmark_coords.append(landmark.y)
            
            # All coordinates should have 42 digits
            if len(landmark_coords) == 42:
                landmarks.append(landmark_coords)
                images.append(image_path)
                labels.append(label)
                labels_idx.append(idx)
    
    return images, labels, labels_idx, landmarks
    
    
X_images, y_labels, y_labels_idx, landmarks = load_data("./data")

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [04:11<00:00,  7.19s/it]


In [4]:
import tensorflow as tf
import pickle

In [5]:
y_labels_cat =sorted(os.listdir("./data"))
X_train, X_test, y_train, y_test = train_test_split(landmarks, y_labels, test_size = 0.3, stratify = y_labels, random_state=10, shuffle = True)

In [6]:
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, chi2  # Example using chi-square test
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# selector = SelectKBest(chi2, k=42)  # Replace chi2 with other feature selection methods if needed
# X_train = selector.fit_transform(X_train, y_train)
# X_test = selector.transform(X_test)
from sklearn.ensemble import RandomForestClassifier


In [7]:
from sklearn.svm import SVC
model = LogisticRegression(solver='liblinear') 
import pickle

model.fit(X_train, y_train)

y_predict = model.predict(X_test)

score = accuracy_score(y_predict, y_test)

print('{}% of samples were classified correctly !'.format(score * 100))

f = open('model.p', 'wb')
pickle.dump({'model': model}, f)
f.close()
pickle.dump(model, open('./model.p', 'wb'))
 
# some time later...
 


81.22605363984674% of samples were classified correctly !


In [12]:
import pickle

import cv2
import mediapipe as mp
import numpy as np

model_dict = pickle.load(open('./model.p', 'rb'))
# model = model_dict['model']

cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5)

labels_dict = {0: 'A', 1: 'B', 2: 'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',16:'R',17:'S',18:'T',19:'U',20:'V',21:'W',22:'X',23:'Y',24:'Z',25:'0',26:'1',27:'2',28:'3',29:'4',30:'5',31:'6',32:'7',33:'8',34:'9'}
while True:

    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())#each finger diff color

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))#normalization
                data_aux.append(y - min(y_))
#bounding box use rec for data_aux landmark var
        x1 = int(min(x_) * W) - 10#bootom right(*W to localize in the hand)
        y1 = int(min(y_) * H) - 10#-10 for big rectangle

        x2 = int(max(x_) * W) - 10#top leftf
        y2 = int(max(y_) * H) - 10

        prediction = model.predict([np.asarray(data_aux)])

        predicted_character = labels_dict[int(prediction[0])]#

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)#coordinates,color,thick
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                    cv2.LINE_AA)#text on top of bounding box

    cv2.imshow('frame', frame)
    key = cv2.waitKey(1)
    if key == ord("q"):
            break


cap.release()
cv2.destroyAllWindows()
#detect only one hand 42 features